In [ ]:
!nvidia-smi # You want 16280MiB, just hardreset runtime and run again.

In [ ]:
%tensorflow_version 1.x

from google.colab import drive
drive.mount('/content/drive')

!mkdir /content/GeneratedDataset
!mkdir /content/GeneratedDataset/GeneratedImagesCurrent
!mkdir /content/GeneratedDataset/GeneratedImagesXmlCurrent
!mkdir /content/GeneratedDataset/GeneratedImagesNew
!mkdir /content/GeneratedDataset/GeneratedImagesXmlNew

!wget https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz
!tar xf dtd-r1.0.1.tar.gz
!rm /content/dtd-r1.0.1.tar.gz

!git clone https://github.com/AIWintermuteAI/aXeleRate.git
!pip uninstall -y imgaug && pip uninstall -y albumentations && pip install imgaug==0.4

%cd /content/
!mkdir /content/Yolo-digit-detector
%cd /content/Yolo-digit-detector

In [ ]:
%cd /content
from keras import backend as K
import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training, setup_inference
import os

In [ ]:
# Satellite sends down I have taken a picture at the time xx: xx: xx
# Lab sends data on all boats' positions at time xx: xx: xx
# Satellite receives the annotations and places them on its "rolling" training set. Pops oldest image and append latest.

In [ ]:
class Annotation_lab:
    def __init__(self, data):
        self.data = data                                                                                # Hold all data of type dictionary TIME : { LABEL: COORD, SIZE }
        self.satelites = []                                                                             # Array of all satelites that whish to comunicate with this lab

    def send_annotation(self, time, label):
        data_from_labels = { }
        for key, value in self.data[time].items():
            if str(key) in label:
                data_from_labels.update({key: value})

        return data_from_labels
        # return self.data[time][label]                                                                   # Sends all annotations of this label at given time

    def add_data(self, data):
        self.data.update(data)                                                                          # Of type dictionary TIME : { LABEL: COORD, SIZE }<

    def add_satelite(self, satelite):
        self.satelites.append(satelite)                                                                 # Adds satelite

    def aggregate_weights(self): # Aggregates all weights from each satelite and sends aggregated weights back
        weighted_sum
        for satelite in self.satelites:
            weighted_sum += satelite.send_weight()

        for satelite in self.satelites:
            satelite.download_weight(weighted_sum)

import xml.etree.cElementTree as ET
import cv2
from keras import backend as K
from axelerate import setup_training, setup_inference
import os
import shutil

import matplotlib.pyplot as plt
from pathlib import Path
import xarray as xr
import numpy as np

class Satelite:
    def get_config(self, model_path, architecture, batch_size, saved_folder, train_times, actual_epoch, learning_rate, labels):
        config = {
            "model":{
                "type":             "Detector",
                "architecture":     architecture,
                "input_size":       2000,         # 123 prolly will have to crop image around annotation
                "anchors":          [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828],
                "labels":           labels,
                "coord_scale" : 		1.0,
                "class_scale" : 		1.0,
                "object_scale" : 		5.0,
                "no_object_scale" : 1.0
            },
            "weights" : {
                "full":   				  model_path,
                "backend":   		    "imagenet"
            },
            "train" : {
                "actual_epoch":         actual_epoch,
                "train_image_folder":   "/content/GeneratedDataset/GeneratedImagesCurrent",
                "train_annot_folder":   "/content/GeneratedDataset/GeneratedImagesXmlCurrent",
                "train_times":          train_times,
                "valid_image_folder":   "/content/GeneratedDataset/GeneratedImagesNew",
                "valid_annot_folder":   "/content/GeneratedDataset/GeneratedImagesXmlNew",
                "valid_times":          1,
                "valid_metric":         "mAP",
                "batch_size":           batch_size,
                "learning_rate":        learning_rate,
                "saved_folder":   	    saved_folder,
                "first_trainable_layer":"",
                "augumentation":				False,
                "is_only_detect" : 		  False
            },
            "converter" : {
                "type":   				["k210","kmodel"]
            }
        }

        return config

    def __init__(self, annotation_lab, train_times_per_dataset, labels, actual_epoch, learning_rate):
        self.annotation_lab = annotation_lab                                                            # Simulated annotation lab that holds all data

        self.labels = labels                                                                            # Labels that we train on

        # self.current_image_dataset_dir = "/content/GeneratedDataset/GeneratedImagesCurrent"             # Folder holding current training dataset
        # self.current_annotation_dataset_dir = "/content/GeneratedDataset/GeneratedImagesXmlCurrent"     # - || -
        # self.train_time = train_times_per_dataset                                                       # After you have trained train_time's delete current_image_dataset_dir and copy over this dataset:
        # self.new_image_dataset_dir = "/content/GeneratedDataset/GeneratedImagesNew"                     # Folder to append new accumulated training data
        # self.new_annotation_dataset_dir = "/content/GeneratedDataset/GeneratedImagesXmlNew"             # - || -

        path_10x10_2 = '/content/drive/MyDrive/agriculture_1_2019_500_500_1000_1000.nc'
        self.nc_dataset = xr.open_dataset(path_10x10_2, chunks={"time": 10})

        self.config = self.get_config("/content/GeneratedDataset", "MobileNet1_0", 128, "/content/GeneratedDataset", train_times_per_dataset, actual_epoch, learning_rate, labels)

    def take_new_image(self, time_stamp):                                                        # Acts that new image was taken
        # Extract image from self.nc_dataset like eric did at this timestamp
        # We have to read in image because we cant take an actual image

        loaded_image = self.nc_dataset[["B04_10m", "B03_10m", "B02_10m"]].isel(time=time_stamp)
        img = loaded_image.to_array().plot.imshow(size=10,vmin=0, vmax=2500)

        image_dir = self.config["train"]["valid_image_folder"] + "/{}.JPEG".format(time_stamp)        # Dir to new dataset        
        
        plt.axis('tight')
        img.set_cmap('hot')
        plt.axis('off')
        plt.axis('off')
        plt.savefig(image_dir, bbox_inches='tight')

        shape = [2000, 2000]

        self.request_annotation(shape, time_stamp)                                              # 

    def request_annotation(self, image_shape, time_stamp):
      #print(image_shape)
      def create_root(file_prefix, image_shape):
          root = ET.Element("annotations")
          ET.SubElement(root, "filename").text = "{}.JPEG".format(file_prefix)
          ET.SubElement(root, "folder").text = "images"
          size = ET.SubElement(root, "size")
          ET.SubElement(size, "width").text = str(image_shape[0])
          ET.SubElement(size, "height").text = str(image_shape[1])
          ET.SubElement(size, "depth").text = "3"
          return root
      
      def create_object_annotation(root, voc_labels):
          for key, value in voc_labels.items():
              for val in value:
                  obj = ET.SubElement(root, "object")
                  ET.SubElement(obj, "name").text = key
                  ET.SubElement(obj, "pose").text = "Unspecified"
                  ET.SubElement(obj, "truncated").text = str(0)
                  ET.SubElement(obj, "difficult").text = str(0)
                  bbox = ET.SubElement(obj, "bndbox")

                  middle = val[0]
                  size = val[1]

                  #print(middle)
                  #print(size)

                  ET.SubElement(bbox, "xmin").text = str(middle[0] - (float(size[0]) * 0.5))
                  ET.SubElement(bbox, "ymin").text = str(middle[1] - (float(size[1]) * 0.5))
                  ET.SubElement(bbox, "xmax").text = str(middle[0] + (float(size[0]) * 0.5))
                  ET.SubElement(bbox, "ymax").text = str(middle[1] + (float(size[1]) * 0.5))
          return root

      root = create_root(time_stamp, image_shape)                                                         # 
      root = create_object_annotation(root, annotation_lab.send_annotation(str(time_stamp), self.labels))      # 
      tree = ET.ElementTree(root)                                                                         # 
      tree.write(self.config["train"]["valid_annot_folder"] + "/{}.xml".format(time_stamp))               # 

      # does not right now process world_coords to pixel_coords, it takes raw data from lab

    # def add_annotation_to_list(label, coord1, coord2):
    #     return [label, coord1, coord2]

    def switch_validation_training(self):
        # switch the training dirs with each other
        # delete old data

        def delete_in_folder(folder):
            for filename in os.listdir(folder):
                file_path = os.path.join(folder, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print('Failed to delete %s. Reason: %s' % (file_path, e))

        train_image_folder = self.config["train"]["train_image_folder"]
        train_annot_folder = self.config["train"]["train_annot_folder"]
        delete_in_folder(train_image_folder)
        delete_in_folder(train_annot_folder)
        
        valid_image_folder = self.config["train"]["valid_image_folder"]
        valid_annot_folder = self.config["train"]["valid_annot_folder"]

        self.config["train"]["train_image_folder"] = valid_image_folder
        self.config["train"]["train_annot_folder"] = valid_annot_folder
        self.config["train"]["valid_image_folder"] = train_image_folder
        self.config["train"]["valid_annot_folder"] = train_annot_folder

    def train(self):
        

        # train
        # K.clear_session()
        # model_path = setup_training(config_dict=self.config)
        print("implementation was not sufficient")


In [ ]:
# Create virtual annotation_lab

data_block = {

}

data = {
    "1": {
        "house": [
        # Coordinate    width-height
            [[92, 10], [20, 20]],
            [[82, 60], [20, 20]]
        ]
    },
    "2": {
        "house": [
        # Coordinate    width-height
            [[92, 10], [20, 20]],
            [[82, 60], [20, 20]]
        ]
    },
    "3": {
        "house": [
        # Coordinate    width-height
            [[92, 10], [20, 20]],
            [[82, 60], [20, 20]]
        ]
    },
    "4": {
        "house": [
        # Coordinate    width-height
            [[92, 10], [20, 20]],
            [[82, 60], [20, 20]]
        ]
    },
    "5": {
        "house": [
        # Coordinate    width-height
            [[92, 10], [20, 20]],
            [[82, 60], [20, 20]]
        ]
    },
    "6": {
        "house": [
        # Coordinate    width-height
            [[92, 10], [20, 20]],
            [[82, 60], [20, 20]]
        ]
    }
}

annotation_lab = Annotation_lab(data_block)
annotation_lab.add_data(data)

print(annotation_lab.send_annotation("1", ["house"]))

In [ ]:
# Create satelite annotation_lab, train_times_per_dataset, labels, actual_epoch, learning_rate
satelite = Satelite(annotation_lab, 5, ["house"], 1, 1e-4)
annotation_lab.add_satelite(satelite)

In [ ]:
# Simulate taking picture on satelite
satelite.take_new_image(1)
satelite.take_new_image(2)

satelite.switch_validation_training()
satelite.take_new_image(3)
satelite.train()

In [ ]:
# Simulate taking picture on satelite
satelite.take_new_image(4)
satelite.take_new_image(5)

satelite.switch_validation_training()
satelite.take_new_image(6)
satelite.train()